In [1]:
!pip install torch==2.5.1 torchaudio==2.5.1 torchvision==0.20.1 transformers==4.46.1 wandb==0.17.3 datasets==3.1.0 accelerate==1.0.1 vllm==0.6.4.post1 ipykernel==6.28.0 ipython==8.20.0 gradio==4.44.0 trl==0.12.0 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 379.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 219.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 344.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 

In [2]:
!nvidia-smi

Mon Mar 17 12:43:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H200                    On  |   00000000:18:00.0 Off |                    0 |
| N/A   24C    P0             72W /  700W |       1MiB / 143771MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
import os
import logging
import warnings
from dataclasses import dataclass, field, asdict
from typing import Optional
from datetime import datetime

import torch
import transformers
from datasets import load_dataset
import trl
from huggingface_hub import HfApi, login


warnings.filterwarnings("ignore", category=FutureWarning)
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


@dataclass
class TrainingConfig:
    model_name: str = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
    train_file_path: str = "simplescaling/s1K-1.1_tokenized"
    block_size: int = 32768
    epochs: int = 5
    micro_batch_size: int = 1
    gradient_accumulation_steps: int = 1
    learning_rate: float = 1e-5
    weight_decay: float = 1e-4
    warmup_ratio: float = 0.05
    #use_fsdp: bool = True  # Enable Fully Sharded Data Parallel
    use_bf16: bool = True  # Mixed precision training
    eval_steps: int = 50
    save_steps: int = 100
    save_only_model: bool = True
    push_to_hub: bool = True  
    hf_repo_name: str = field(default_factory=lambda: f"Nika-7b")
    output_dir: str = field(default_factory=lambda: f"ckpts/{datetime.now().strftime('%Y%m%d_%H%M%S')}")

    def setup_env(self):
        os.environ["WANDB_PROJECT"] = "nika-7b"
        os.environ["WANDB_ENTITY"] = "beastgokul4-"


def train(config: TrainingConfig):
    config.setup_env()
    logging.info(f"Training config: {asdict(config)}")

    token = os.getenv("HF_TOKEN")  
    if not token:
        raise ValueError("Please set HF_TOKEN in your environment variables.")
    login(token=token)

    
    dataset = load_dataset(config.train_file_path)
    
    model_kwargs = {"device_map": "auto", "torch_dtype": "auto"} if "70B" in config.model_name else {}
    model = transformers.AutoModelForCausalLM.from_pretrained(config.model_name, **model_kwargs)
    

    tokenizer = transformers.AutoTokenizer.from_pretrained(config.model_name, use_fast=True)
    if "Qwen" in config.model_name:
        instruction_template = "<|im_start|>user"
        response_template = "<|im_start|>assistant\n"
        tokenizer.pad_token = "<|fim_pad|>"
    else:
        instruction_template = "<|start_header_id|>user<|end_header_id|>"
        response_template = "<|start_header_id|>assistant<|end_header_id|>\n\n"
        tokenizer.pad_token = "<|reserved_special_token_5|>"

    collator = trl.DataCollatorForCompletionOnlyLM(
        instruction_template=instruction_template,
        response_template=response_template,
        tokenizer=tokenizer,
        mlm=False
    )


    training_args = transformers.TrainingArguments(
        output_dir=config.output_dir,
        per_device_train_batch_size=config.micro_batch_size,
        per_device_eval_batch_size=config.micro_batch_size,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        num_train_epochs=config.epochs,
        logging_steps=1,
        save_steps=config.save_steps,
        evaluation_strategy="steps",
        eval_steps=config.eval_steps,
        learning_rate=config.learning_rate,
        weight_decay=config.weight_decay,
        warmup_ratio=config.warmup_ratio,
        save_total_limit=1,
        push_to_hub=config.push_to_hub,
        hub_model_id=config.hf_repo_name,
        hub_private_repo=True,  
        bf16=config.use_bf16,
        #fsdp="full_shard auto_wrap" if config.use_fsdp else None
    )

    trainer = trl.SFTTrainer(
        model=model,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"] if "test" in dataset else dataset["train"],
        args=training_args,
        data_collator=collator
    )

    trainer.train()

    trainer.save_model(config.output_dir)
    tokenizer.save_pretrained(config.output_dir)
    trainer.accelerator.wait_for_everyone()


    if config.push_to_hub:
        api = HfApi()
        logging.info(f"Uploading model to Hugging Face Hub: {config.hf_repo_name}")
        api.create_repo(config.hf_repo_name, exist_ok=True)
        api.upload_folder(
            folder_path=config.output_dir,
            repo_id=config.hf_repo_name
        )
        logging.info("Model successfully uploaded to Hugging Face!")


if __name__ == "__main__":
    train(TrainingConfig())


2025-03-17 12:43:54,838 - INFO - Training config: {'model_name': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', 'train_file_path': 'simplescaling/s1K-1.1_tokenized', 'block_size': 32768, 'epochs': 5, 'micro_batch_size': 1, 'gradient_accumulation_steps': 1, 'learning_rate': 1e-05, 'weight_decay': 0.0001, 'warmup_ratio': 0.05, 'use_bf16': True, 'eval_steps': 50, 'save_steps': 100, 'save_only_model': True, 'push_to_hub': True, 'hf_repo_name': 'Nika-7b', 'output_dir': 'ckpts/20250317_124354'}
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
2025-03-17 12:43:55,359 - WARNING - Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


README.md:   0%|          | 0.00/665 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/35.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
2025-03-17 12:48:44,795 - ERROR - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: beastgokul4 (beastgokul4-). Use `wandb login --relogin` to force relogin
